# Check Retention Score Freshness
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Apr 30, 2022

In [19]:
import boto3 
import json
import pandas as pd
import psycopg2
import pytz

from datetime import datetime, timedelta
from shared_utilities import helpers

In [20]:
! aws sso login --profile Stellaralgo-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

VNTB-QTJC
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/


In [21]:
session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')

### Let's create a list of team db's that we'd like to check for fresh scores:

In [22]:
teams = json.load(open('../config/teamsconfig.json'))

results = []
for team in teams:

    cnxn = helpers.get_redshift_connection(team['cluster'], team['dbname'])
    cursor = cnxn.cursor()

    sql = f"""
        SELECT count(*), lkupclientid, MAX(scoredate) as max_score_date
        FROM {team['dbname'].lower()}.dw.customerretentionscores 
        WHERE lkupclientid = {team['lkupclientid']}
        GROUP BY lkupclientid
    """

    cursor.execute(sql)

    df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
    df["cluster"] = team["cluster"]
    df["dbname"] = team["dbname"]
    df["lkupclientid"] = team["lkupclientid"]
    df["name"] = team["name"]
    
    if len(df) > 0:
        results.append(df)
    else:
        print(f" > NO RETENTION SCORES FOUND FOR: {team['name']} - {team['clientcode']} ({team['lkupclientid']})")

df_results = pd.concat(results)

df_results["count"] = pd.to_numeric(df_results["count"])
df_results["max_score_date"] = pd.to_datetime(df_results["max_score_date"])

df_results.shape

(38, 6)

### Now we can loop through each datatbase, connect to it and check the retention scores:

In [23]:
today = datetime.now(pytz.timezone("MST")).strftime("%Y-%m-%d")

days_to_sub = 0
target_day = (datetime.today() - timedelta(days_to_sub)).strftime("%Y-%m-%d")

df_stale = df_results[df_results["max_score_date"] < target_day]
df_stale = df_stale.sort_values(by='name')

print(f"STALE TEAMS: {len(df_stale)}")
print(f"TARGET DAY: {target_day}")
df_stale

STALE TEAMS: 38
TARGET DAY: 2022-08-29


,count,lkupclientid,max_score_date,cluster,dbname,name
0,623356,55,2022-08-28,prod-app,stlrcoyotes,Arizona Coyotes
0,1216705,36,2022-08-28,prod-app,stlrflames,Calgary Flames
0,379094,37,2022-08-28,prod-app,stlrwhl,Calgary Hitmen
0,793813,8,2022-08-28,prod-app,stlrnll,Calgary Roughnecks
0,1369225,35,2022-08-28,prod-app,stlrcfl,Calgary Stampeders
0,162562,28,2022-08-28,prod-app,stlrmilb,Columbia Fireflies
0,1785094,11,2022-08-28,prod-app,stlrmilb,Durham Bulls
0,396345,30,2022-08-28,prod-app,stlrmilb,El Paso Chihauhaus
0,72334,99,2022-08-28,prod-app-elbu,stlrusllocomotive,El Paso Locomotive
0,61215,96,2022-08-28,prod-app-elbu,stlrechleverblades,Florida Everblades


### Done